# Exercice 2

**Question 1)**: Pour tout vecteur $ z \in \mathbb{R} $, on note $ L(z) $ le vecteur $ (L_1(z), \dots, L_n(z)) $ dont la composante $ L_k(z) $ est définie par 

$L_k(z) = \sum_{i=1}^{k} z(i)$.

Expliquer pourquoi $ L_k(z) $ est la valeur optimale du programme linéaire en variables 0-1 suivant :

$\min \sum_{i=1}^{n} a_{ik} z_i$

sous les contraintes :

$\sum_{i=1}^{n} a_{ik} = k$,

$a_{ik} \in \{0, 1\}, \quad i = 1, \dots, n$.

**Explication**: Étant donné que le vecteur $ (z_{(1)x}, \ldots, z_{(n)x}) $ est ordonné en ordre croissant, la somme $ \sum_{i=1}^{k} z(i) $ correspond à la somme des $ k $ plus petites valeurs de $ z_i $ et donc à la valeur optimale du programme linéaire.



**Question 2)**: 

Primal:

\begin{align*}
\min & \sum_{i=1}^{n} a_{ik} z_i \\
\text{s.c.} \quad & \sum_{i=1}^{n} a_{ik} = k \\
& 0 \leq a_{ik} \leq 1, \quad i = 1, \dots, n
\end{align*}

Dual:

\begin{align*}
\max & \quad k \cdot r_k - \sum_{i=1}^{n} b_{ik} \\
\text{s.c.} \quad & r_k - b_{ik} \leq z_i, \quad \forall i = 1, \dots, n \\
& b_{ik} \geq 0, \quad \forall i = 1, \dots, n
\end{align*}

Trouver $L(2,9,6,8,5,4)$:

In [47]:
from gurobipy import *

In [48]:
z = [2,9,6,8,5,4]
n = len(z)

In [49]:
for k in range(1,n+1):
    model = Model()
    model.params.LogToConsole = False

    rk = model.addVar(vtype=GRB.CONTINUOUS)
    bki = model.addVars(n,vtype=GRB.CONTINUOUS)

    model.setObjective(k * rk - bki.sum(), GRB.MAXIMIZE)

    model.addConstrs(((rk-bki[i])<=z[i] for i in range(n)), name='c1')
    model.addConstrs((bki[i]>=0 for i in range(n)), name='c2')

    model.optimize()
    composante = int(model.objVal)
    print(f"L_{k}(z) = {composante}")



L_1(z) = 2
L_2(z) = 6
L_3(z) = 11
L_4(z) = 17
L_5(z) = 25
L_6(z) = 34


**Question 3:** Montrer que 
\begin{align*}
g(x)& =\sum_{i=1}^{n}w_iz_{(i)}(x)\\
&=\sum_{k=1}^{n}w'_k L_k(z(x))
\end{align*}
**Explication:** 

\begin{align*}
\text{par définition: }z_{(k)}(x)&=L_k(z(x)) - L_{k-1}(z(x))\\
g(x)& =\sum_{i=1}^{n}w_iz_{(i)}(x)\\
&=w_1z_{(1)}+w_2z_{(2)}+\ldots+w_nz_{(n)}\\
&=w_1L_1(z(x))+w_2(L_2(z(x))-L_1(z(x)))+\ldots+w_n(L_n(z(x))-L_{n-1}(z(x))\\
&=(w_1-w_2)L_1(z(x))+(w_2-w_3)(L_2(z(x)))+\ldots+(w_{n-1}-w_{n})L_n\\
&=\sum^n_{k=1}w'_kL_k(z(x))
\end{align*}

**Question 4:**
\begin{align*}
\max &\sum_{k=1}^n w'_k \left( k \cdot r_k - \sum_{i=1}^n b_{ik} \right) \\
\text{s.c.} & \quad r_k - b_{ik} \leq z_i(x), \quad i = 1, \dots, n \\
& \quad b_{ik} \geq 0, \quad i = 1, \dots, n \\
& \quad x \in X
\end{align*}

$x \in X$ -> solution réalisables ?

In [50]:
model = Model()
model.params.LogToConsole = True


# projets de l'exemple 1
c = 100
valeurs_scenarios = [[70,18,16,14,12,10,8,6,4,2],
                    [2,4,6,8,10,12,14,16,18,70]]
couts = [60,10,15,20,25,20,5,15,20,60]
C_max = 100

n_projets = len(valeurs_scenarios[0])
n_scenarios = len(valeurs_scenarios)

x = model.addVars(n_projets,vtype=GRB.BINARY,name='x')
z = []
for i in range(n_scenarios):
    z.append(quicksum([valeurs_scenarios[i][j]*x[j] for j in range(n_projets)]))

w = [2,1]
n = len(w)

w_apostrophe = [w[k] - w[k+1] for k in range(len(w)-1)]
w_apostrophe.append(w[-1])
rk = model.addVars(n,vtype=GRB.CONTINUOUS, name='rk')
bik = []
for k in range(n):
    bik.append(model.addVars(n,vtype=GRB.CONTINUOUS, lb=0,name=f'bik {k}'))
print(w_apostrophe)


[1, 1]


In [51]:
objectif = quicksum([w_apostrophe[k]*(k*rk[k]-quicksum(bik[k][i] for i in range(n))) for k in range(n)])
model.setObjective(objectif, GRB.MAXIMIZE)
for k in range(n):
    for i in range(n):
        model.addConstr(rk[k]-bik[k][i]<=z[i]) # premiere contrainte
        model.addConstr(bik[k][i]>=0)

model.addConstr(quicksum(couts[j] * x[j] for j in range(n_projets)) <= C_max, name="BudgetConstraint")


<gurobi.Constr *Awaiting Model Update*>

In [52]:
model.optimize()
model.ObjVal

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "elementary OS 7.1 Horus")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 16 columns and 62 nonzeros
Model fingerprint: 0xf46fdb5e
Variable types: 6 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolved: 3 rows, 13 columns, 34 nonzeros
Variable types: 3 continuous, 10 integer (10 binary)

Root relaxation: objective 7.700000e+01, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

66.0

In [53]:
objets_selectiones = [i for i in range(n_projets) if x[i].X > 0.5]
objets_selectiones

[1, 2, 3, 6, 7, 8]

In [54]:
model.ObjVal

66.0

**Question 5:** trouver un formulation linéaire pour le critère de *minOWA*

**Réponse:** 

Primal:
\begin{align*}
    L_k(z)&=\sum_{i=1}^{k} r(x,s_{(i)}) \text{ regret en ordre décroissant, donc:}\\
    \max &\sum_{i=1}^n a_{ik} r(x,s_{(i)})\\
\text{s.c.} \quad & \sum_{i=1}^{n} a_{ik} = k \\
& 0 \leq a_{ik} \leq 1, \quad i = 1, \dots, n
\end{align*}
$r(x,s_{(i)}) = z_i^*-z_i(x)$

Dual:
\begin{align*}
\min & \quad k \cdot r_k + \sum_{i=1}^{n} b_{ik} \\
\text{s.c.} \quad & r_k - b_{ik} \geq r(x,s_{(i)}), \quad \forall i = 1, \dots, n \\
& b_{ik} \geq 0, \quad \forall i = 1, \dots, n
\end{align*}

Formulation entière:

\begin{align*}
\min &\sum_{k=1}^n w'_k \left( k \cdot r_k + \sum_{i=1}^n b_{ik} \right) \\
\text{s.c.} & \quad r_k - b_{ik} \geq r(x,s_{(i)}), \quad i = 1, \dots, n \\
& \quad b_{ik} \geq 0, \quad i = 1, \dots, n \\
& \quad x \in X
\end{align*}

In [55]:
# trouver z*

def model_sad_maxmin(c,valeurs_scenarios, p):
    n = len(p)
    model = Model()
    model.params.LogToConsole = False

    t = model.addVar(vtype = GRB.INTEGER)
    x = model.addVars(n, vtype=GRB.BINARY)

    model.setObjective(t, GRB.MAXIMIZE)

    model.addConstr(quicksum(p[i] * x[i] for i in range(n)) <= c) # contraint de poids maximum

    for s_p in valeurs_scenarios:
        model.addConstr(t <= quicksum(s_p[i] * x[i] for i in range(n))) # contraint de valeur minimale pour scénario s

    model.optimize()
    objets_selectiones = [i for i in range(n) if x[i].X > 0.5]
    profit = int(model.objVal)

    vecteur_image_z = []
    for v_s in valeurs_scenarios:
        vecteur_image_z.append(sum([v for i,v in enumerate(v_s) if i in objets_selectiones]))

    return {'vecteur_z':vecteur_image_z, 'objets': objets_selectiones, 'profit': profit}

z_etoile = []
for v_s in valeurs_scenarios:
    gurobi_solution = model_sad_maxmin(c=C_max, valeurs_scenarios=[v_s], p=couts) # model_sad_maxmin avec un scénario est égal à optimiser le problème sad
    z_etoile.append(gurobi_solution['profit'])
z_etoile

[112, 118]

In [56]:
model = Model()
model.params.LogToConsole = True

n_projets = len(valeurs_scenarios[0])
n_scenarios = len(valeurs_scenarios)

x = model.addVars(n_projets,vtype=GRB.BINARY,name='x')

w = [2,1]
n = len(w)

# calculer le regret r(x,s(i))
r = []
for i in range(n_scenarios):
    z = quicksum([valeurs_scenarios[i][j]*x[j] for j in range(n_projets)])
    r.append(z_etoile[i]-z)

w_apostrophe = [w[k] - w[k+1] for k in range(len(w)-1)]
w_apostrophe.append(w[-1])
rk = model.addVars(n,vtype=GRB.CONTINUOUS, name='rk')
bik = []
for k in range(n):
    bik.append(model.addVars(n,vtype=GRB.CONTINUOUS, lb=0,name=f'bik {k}'))
print(w_apostrophe)

objectif = quicksum([w_apostrophe[k]*(k*rk[k]+quicksum(bik[k][i] for i in range(n))) for k in range(n)])
model.setObjective(objectif, GRB.MINIMIZE)
for k in range(n):
    for i in range(n):
        model.addConstr(rk[k]-bik[k][i]>=r[i]) # premiere contrainte
        model.addConstr(bik[k][i]>=0)

model.addConstr(quicksum(couts[j] * x[j] for j in range(n_projets)) <= C_max, name="BudgetConstraint")



[1, 1]


<gurobi.Constr *Awaiting Model Update*>

In [57]:
model.optimize()
model.ObjVal

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "elementary OS 7.1 Horus")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 16 columns and 62 nonzeros
Model fingerprint: 0x1b901db1
Variable types: 6 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 118.0000000
Presolve removed 6 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 11 columns, 32 nonzeros
Variable types: 0 continuous, 11 integer (10 binary)
Found heuristic solution: objective 116.0000000

Root relaxation: objective 3.800000e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incu

50.0

In [58]:
objets_selectiones = [i for i in range(n_projets) if x[i].X > 0.5]
objets_selectiones


[1, 2, 5, 6, 7, 8]